In [2]:
import numpy as np
import pandas as pd

ratings = pd.read_csv('../data/ratings.csv')
anime = pd.read_csv('../data/animes_data.csv')

In [3]:
anime_id = anime["anime_id"].tolist()

In [4]:
ratings = ratings[ratings["anime_id"].isin(anime_id)] #メタデータに含まれるアニメのみ

In [5]:
ratings = ratings[ratings["rating"] >= 8] #閾値設定

In [6]:
ratings['user_id'].value_counts()

user_id
64481     891
102523    863
80494     848
62683     844
2809      837
         ... 
20270       1
31017       1
5142        1
57030       1
92953       1
Name: count, Length: 81903, dtype: int64

In [9]:
counts = ratings['user_id'].value_counts(sort=True)
person = counts[counts >= 10].keys() #10個以上の評価値を持っていないユーザーを削除

In [10]:
ratings = ratings[ratings['user_id'].isin(person)]

In [11]:
round(ratings.describe(),2)

,user_id,anime_id,rating
count,5241100.00,5241100.00,5241100.00
mean,54113.39,14556.47,8.79
std,32016.05,12767.25,0.80
min,1.00,1.00,8.00
25%,25542.00,2418.00,8.00
50%,55034.00,10863.00,9.00
75%,82056.00,25687.00,9.00
max,108024.00,48456.00,10.00


In [32]:
# ユーザIDをキーが、コンテキストが値
item_buckets = {}

for user_id in ratings['user_id'].unique():
    # ユーザごとに、評価が平均値以上のアイテムIDを抽出して' 'で連結する
    #average = np.average(ratings[ratings['user_id'] == user_id]['rating'])
    #user_ratings = ratings[(ratings['user_id'] == user_id) & (ratings['rating'] >= average)]
    user_ratings = ratings[(ratings['user_id'] == user_id)]
    item_buckets[user_id] = ' '.join(user_ratings['anime_id'].astype('str'))

print('item_buckets[1]: {}'.format(item_buckets[1]))
# item_buckets[1]: 61 33 160 20 202 171 265 47 222 253 113 227 90 64 228 121 114 132 134 98 186 221 84 60 177 174 82 56 80 229 235 6 206 76 72 185 96 258 81 212 151 51 175 107 209 108 12 14 44 163 210 184 157 150 183 248 208 128 242 193 236 250 91 129 241 267 86 196 39 230 23 224 65 190 100 154 214 161 170 9 246 22 187 135 68 146 176 166 89 249 269 32 270 133 239 194 256 93 234 1 197 173 75 268 144 119 181 257 109 182 223 46 169 162 66 77 199 57 50 192 178 87 238 156 106 115 137 127 16 79 45 48 25 251 195 168 123 191 203 55 42 7 43 165 198 124 95 58 216 204 3 207 19 18 59 15 111 52 88 13 28 172 152

# # ml100k.txtファイルに書き出す
# with open('ml100k.txt', 'w') as f:
#     f.write('\n'.join(item_buckets.values()))

item_buckets[1]: 36904 9875 9989 37171 13535 11633 36456 16405 6280 10029 31636 26349 11843 14353 1535 552 2961 37520 223 225 813 894 895 14837 6746 8408 317 2952 6151 7902 395 918 7472 15335 9969 15417 2951 9735 6945 4872 245 1824 1030 578 34933 37086 18679 21659 32281 7593 15379 14349 512 96 80 2581 6288 1090 1091 1092 3001 4081 1210 20507 7592 464 30276 1060 8795 437 10588 13587 326 527 14123 528 1117 34540 5526 4792 20159 30240 33255 34612 38249 13055 2605 3674 199 5355 7724 25777 38524 9863 3588 9253 10863 5681 849 4382 9941 523 5365 13171 16742 19383 10216 6377 7044 5300 3228


In [33]:
# txtファイルに書き出す
# with open('../data/myanimelist_user.txt', 'w') as f:
#     f.write('\n'.join(item_buckets.values()))

In [37]:
from gensim.models import word2vec


sentences = word2vec.LineSentence('../data/myanimelist_user.txt')

# word2vecのモデルを構築
#     sg: skip-gram
#     size: 300次元
#     window: 1000 ユーザーの評価数がmax891であるため
#     hs: ネガティブサンプリング
#     negative: デフォルト5
#     seed: 0に設定
model = word2vec.Word2Vec(sentences, sg=1, vector_size =300, window=1000, hs=0, negative=5, seed=0)

#約23分

In [38]:
# model.wv.save_word2vec_format('../model/myanimelist_user_word2vec.txt')

In [12]:
from gensim.models import KeyedVectors

model = KeyedVectors.load_word2vec_format("../model/myanimelist_user_word2vec.txt")

In [13]:
model.most_similar(positive='849', topn=10)

[('4382', 0.7017595171928406),
 ('7311', 0.6571195125579834),
 ('1210', 0.6544589400291443),
 ('2236', 0.6476010084152222),
 ('11577', 0.6386355757713318),
 ('10165', 0.6318712830543518),
 ('13125', 0.6237733960151672),
 ('10721', 0.6138961315155029),
 ('5341', 0.6099304556846619),
 ('10863', 0.6076909303665161)]

In [14]:
# 「アイテムID : 映画タイトル : 類似度」を順番に表示する
def print_similar_titles(tuples):
    for t in tuples:
        print('{} : {} : {}'.format(t[0], anime['ja_title'][anime["anime_id"]==int(t[0])].iloc[0], t[1]))

In [17]:
anime[anime['ja_title'].str.contains("ドラゴンボール")]

,anime_id,title,main_picture,ja_title,synopsis,genres,media,episodes,rating,members,start_date,season,source,studios
174,223,Dragon Ball,https://api-cdn.myanimelist.net/images/anime/1...,ドラゴンボール,Gokuu Son is a young boy who lives in the wood...,"Action,Adventure,Comedy,Fantasy,Martial Arts,S...",tv,153,7.96,985273,1986-02-26,winter_1986,manga,Toei Animation
175,225,Dragon Ball GT,https://api-cdn.myanimelist.net/images/anime/1...,ドラゴンボールGT,Emperor Pilaf finally has his hands on the Bla...,"Action,Adventure,Comedy,Fantasy,Martial Arts,S...",tv,64,6.48,605582,1996-02-07,winter_1996,manga,Toei Animation
369,502,Dragon Ball Movie 1: Shen Long no Densetsu,https://api-cdn.myanimelist.net/images/anime/1...,ドラゴンボール 神龍の伝説,"A retelling of Dragon Ball's origins, this is ...","Action,Adventure,Comedy,Fantasy,Shounen,Super ...",movie,1,6.77,65253,1986-12-20,fall_1986,manga,Toei Animation
511,813,Dragon Ball Z,https://api-cdn.myanimelist.net/images/anime/1...,ドラゴンボールZ,Five years after winning the World Martial Art...,"Action,Adventure,Comedy,Fantasy,Martial Arts,S...",tv,291,8.16,1172492,1989-04-26,spring_1989,manga,Toei Animation
563,891,Dragon Ball Movie 2: Majinjou no Nemurihime,https://cdn.myanimelist.net/images/anime/1220/...,ドラゴンボール 魔神城のねむり姫,Goku and Kuririn are given an assignment by Ka...,"Action,Adventure,Comedy,Fantasy,Sci-Fi,Shounen...",movie,1,6.74,61819,1987-07-18,summer_1987,manga,Toei Animation
564,892,Dragon Ball Movie 3: Makafushigi Daibouken,https://api-cdn.myanimelist.net/images/anime/1...,ドラゴンボール 摩訶不思議大冒険,"Emperor Chiaotzu's wife has gone missing, and ...","Action,Adventure,Comedy,Fantasy,Sci-Fi,Shounen...",movie,1,6.82,55309,1988-07-09,summer_1988,manga,Toei Animation
565,893,Dragon Ball Movie 4: Saikyou e no Michi,https://cdn.myanimelist.net/images/anime/1497/...,ドラゴンボール 最強への道,A retelling of Dragon Ball's origin with a dif...,"Action,Adventure,Comedy,Fantasy,Sci-Fi,Shounen...",movie,1,7.12,59063,1996-03-02,winter_1996,manga,Toei Animation
566,894,Dragon Ball Z Movie 01: Ora no Gohan wo Kaese!!,https://api-cdn.myanimelist.net/images/anime/1...,ドラゴンボールZ オラの悟飯をかえせッ!!,Piccolo is training at a barren cliff when a h...,"Action,Adventure,Comedy,Fantasy,Sci-Fi,Shounen",movie,1,6.64,118376,1989-07-15,summer_1989,manga,Toei Animation
567,895,Dragon Ball Z Movie 02: Kono Yo de Ichiban Tsu...,https://api-cdn.myanimelist.net/images/anime/1...,ドラゴンボールZ この世で一番強いヤツ,"In his laboratory, biotechnology expert Dr. Ko...","Action,Adventure,Comedy,Fantasy,Sci-Fi,Shounen",movie,1,6.59,109147,1990-03-10,winter_1990,manga,Toei Animation
568,896,Dragon Ball Z Movie 03: Chikyuu Marugoto Chouk...,https://api-cdn.myanimelist.net/images/anime/1...,ドラゴンボールZ 地球まるごと超決戦,"A mysterious device crashes on planet Earth, c...","Action,Adventure,Comedy,Fantasy,Sci-Fi,Shounen",movie,1,6.69,131122,1990-07-07,summer_1990,manga,Toei Animation


In [18]:
myanime = ["16498", "1735", "24405", "31240", "14719"]
dislike = []

In [19]:
print_similar_titles(model.most_similar(positive=myanime, negative=dislike, topn=10))

30240 : 監獄学園〈プリズンスクール〉 : 0.7206149697303772
32282 : 食戟のソーマ 弍ノ皿 : 0.7034047842025757
31741 : マギ シンドバッドの冒険 : 0.7019414305686951
34577 : 七つの大罪 戒めの復活 : 0.7013494372367859
23755 : 七つの大罪 : 0.6883667707443237
28623 : 甲鉄城のカバネリ : 0.6878124475479126
30503 : ノラガミ ARAGOTO : 0.686297595500946
29803 : オーバーロード : 0.6834833025932312
32182 : モブサイコ100 : 0.6823713183403015
30276 : ワンパンマン : 0.6823193430900574


In [20]:
anime[anime['anime_id'] == 5680]

,anime_id,title,main_picture,ja_title,synopsis,genres,media,episodes,rating,members,start_date,season,source,studios
1406,5680,K-On!,https://api-cdn.myanimelist.net/images/anime/1...,けいおん!,A fresh high school year always means much to ...,"CGDCT,Comedy,Music,School",tv,13,7.85,1011863,2009-04-03,spring_2009,4_koma_manga,Kyoto Animation
